In [13]:
!apt-get update

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Hit:2 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Get:3 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Ign:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:10 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Get:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.com/ubuntu 

In [14]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/nul

In [15]:
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz

In [17]:
!ls

1HFMUbUU56hvaU-6c0PsZ6ogpiatR_R4y  gdrive	spark-3.1.1-bin-hadoop3.2
drive				   sample_data	spark-3.1.1-bin-hadoop3.2.tgz


In [16]:
!tar xf spark-3.1.1-bin-hadoop3.2.tgz

In [18]:
!pip install -q findspark

In [19]:
import os
os.environ["JAVA_HOME"]= "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

In [20]:
pwd

'/content'

In [21]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local").appName("Lab1").getOrCreate()
spark

In [25]:
!wget -p /content/drive/MyDrive/data/insurance.csv

/content/drive/MyDrive/data/insurance.csv: Scheme missing.


In [26]:
!ls

1HFMUbUU56hvaU-6c0PsZ6ogpiatR_R4y    sample_data
1HFMUbUU56hvaU-6c0PsZ6ogpiatR_R4y.1  spark-3.1.1-bin-hadoop3.2
drive				     spark-3.1.1-bin-hadoop3.2.tgz
gdrive


In [31]:
df = spark.read.csv("/content/drive/MyDrive/data/insurance.csv", sep=",", header=True)

In [32]:
df.show(5, truncate = False)

+---+------+------+--------+------+---------+-----------+
|age|sex   |bmi   |children|smoker|region   |charges    |
+---+------+------+--------+------+---------+-----------+
|19 |female|27.9  |0       |yes   |southwest|16884.924  |
|18 |male  |33.77 |1       |no    |southeast|1725.5523  |
|28 |male  |33    |3       |no    |southeast|4449.462   |
|33 |male  |22.705|0       |no    |northwest|21984.47061|
|32 |male  |28.88 |0       |no    |northwest|3866.8552  |
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows



In [33]:
df.columns

['age', 'sex', 'bmi', 'children', 'smoker', 'region', 'charges']

In [34]:
df.dtypes

[('age', 'string'),
 ('sex', 'string'),
 ('bmi', 'string'),
 ('children', 'string'),
 ('smoker', 'string'),
 ('region', 'string'),
 ('charges', 'string')]

In [36]:
from traitlets.traitlets import TraitError
df = spark.read.csv("/content/drive/MyDrive/data/insurance.csv", sep=",",  header=True, inferSchema=True)

In [37]:
df.dtypes

[('age', 'int'),
 ('sex', 'string'),
 ('bmi', 'double'),
 ('children', 'int'),
 ('smoker', 'string'),
 ('region', 'string'),
 ('charges', 'double')]

In [38]:
df.printSchema()

root
 |-- age: integer (nullable = true)
 |-- sex: string (nullable = true)
 |-- bmi: double (nullable = true)
 |-- children: integer (nullable = true)
 |-- smoker: string (nullable = true)
 |-- region: string (nullable = true)
 |-- charges: double (nullable = true)



In [43]:
from pyspark.sql.types import *

lables = [
    ('age', IntegerType()),
    ('sex', StringType()),
    ('bmi', DoubleType()),
    ('children', IntegerType()),
    ('smoker', StringType()),
    ('region', StringType()),
    ('charges', DoubleType())
    
]

In [44]:
schema = StructType([StructField(x[0], x[1]) for x in lables])
schema

StructType(List(StructField(age,IntegerType,true),StructField(sex,StringType,true),StructField(bmi,DoubleType,true),StructField(children,IntegerType,true),StructField(smoker,StringType,true),StructField(region,StringType,true),StructField(charges,DoubleType,true)))

In [46]:
df = spark.read.csv("/content/drive/MyDrive/data/insurance.csv", header=True,sep=",",  schema = schema, inferSchema = True)
df.show(5)

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
+---+------+------+--------+------+---------+-----------+
only showing top 5 rows



In [47]:
df.show()

+---+------+------+--------+------+---------+-----------+
|age|   sex|   bmi|children|smoker|   region|    charges|
+---+------+------+--------+------+---------+-----------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|
| 33|  male|22.705|       0|    no|northwest|21984.47061|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107|
| 60|female| 25.84|       0|    no|northwest|28923.13692|
| 25|  male| 26.22|       0|    no|northeast|  2721.3208|
| 62|female| 26.29|       0|   yes|southeast| 27808.7251|
| 23|  male|  34.4|       0|    no|southwest|   1826.843|
| 56|female| 39.82|       0|    no|southeast| 11090.7178|
| 27|  male| 4

In [48]:
df.select(df['age']).show(truncate=False)

+---+
|age|
+---+
|19 |
|18 |
|28 |
|33 |
|32 |
|31 |
|46 |
|37 |
|37 |
|60 |
|25 |
|62 |
|23 |
|56 |
|27 |
|19 |
|52 |
|23 |
|56 |
|30 |
+---+
only showing top 20 rows



In [49]:
df.select(df.age, df.sex, df.bmi).show(truncate=False)

+---+------+------+
|age|sex   |bmi   |
+---+------+------+
|19 |female|27.9  |
|18 |male  |33.77 |
|28 |male  |33.0  |
|33 |male  |22.705|
|32 |male  |28.88 |
|31 |female|25.74 |
|46 |female|33.44 |
|37 |female|27.74 |
|37 |male  |29.83 |
|60 |female|25.84 |
|25 |male  |26.22 |
|62 |female|26.29 |
|23 |male  |34.4  |
|56 |female|39.82 |
|27 |male  |42.13 |
|19 |male  |24.6  |
|52 |female|30.78 |
|23 |male  |23.845|
|56 |male  |40.3  |
|30 |male  |35.3  |
+---+------+------+
only showing top 20 rows



In [50]:
from pyspark.sql.functions import col
df.select(col('age')).show(truncate=False)

+---+
|age|
+---+
|19 |
|18 |
|28 |
|33 |
|32 |
|31 |
|46 |
|37 |
|37 |
|60 |
|25 |
|62 |
|23 |
|56 |
|27 |
|19 |
|52 |
|23 |
|56 |
|30 |
+---+
only showing top 20 rows



In [51]:
df.select(df.age).show(truncate=False)

+---+
|age|
+---+
|19 |
|18 |
|28 |
|33 |
|32 |
|31 |
|46 |
|37 |
|37 |
|60 |
|25 |
|62 |
|23 |
|56 |
|27 |
|19 |
|52 |
|23 |
|56 |
|30 |
+---+
only showing top 20 rows



In [55]:
from pyspark.sql.functions import lit,concat
df=df.withColumn('first_name',lit("new")).withColumn('second_column',concat(col('region'), lit(' '), col('bmi')))
df.show()

+---+------+------+--------+------+---------+-----------+----------+----------------+
|age|   sex|   bmi|children|smoker|   region|    charges|first_name|   second_column|
+---+------+------+--------+------+---------+-----------+----------+----------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|       new|  southwest 27.9|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|       new| southeast 33.77|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|       new|  southeast 33.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|       new|northwest 22.705|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|       new| northwest 28.88|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|       new| southeast 25.74|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|       new| southeast 33.44|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|       new| northwest 27.74|
| 37|  male| 29.83|       2|    no|northeast|  6406.41

In [56]:
df=df.withColumnRenamed('second_column','Region||BMI')
df.show()

+---+------+------+--------+------+---------+-----------+----------+----------------+
|age|   sex|   bmi|children|smoker|   region|    charges|first_name|     Region||BMI|
+---+------+------+--------+------+---------+-----------+----------+----------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|       new|  southwest 27.9|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|       new| southeast 33.77|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|       new|  southeast 33.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|       new|northwest 22.705|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|       new| northwest 28.88|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|       new| southeast 25.74|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|       new| southeast 33.44|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|       new| northwest 27.74|
| 37|  male| 29.83|       2|    no|northeast|  6406.41

In [57]:
df.collect()

[Row(age=19, sex='female', bmi=27.9, children=0, smoker='yes', region='southwest', charges=16884.924, first_name='new', Region||BMI='southwest 27.9'),
 Row(age=18, sex='male', bmi=33.77, children=1, smoker='no', region='southeast', charges=1725.5523, first_name='new', Region||BMI='southeast 33.77'),
 Row(age=28, sex='male', bmi=33.0, children=3, smoker='no', region='southeast', charges=4449.462, first_name='new', Region||BMI='southeast 33.0'),
 Row(age=33, sex='male', bmi=22.705, children=0, smoker='no', region='northwest', charges=21984.47061, first_name='new', Region||BMI='northwest 22.705'),
 Row(age=32, sex='male', bmi=28.88, children=0, smoker='no', region='northwest', charges=3866.8552, first_name='new', Region||BMI='northwest 28.88'),
 Row(age=31, sex='female', bmi=25.74, children=0, smoker='no', region='southeast', charges=3756.6216, first_name='new', Region||BMI='southeast 25.74'),
 Row(age=46, sex='female', bmi=33.44, children=1, smoker='no', region='southeast', charges=8240.

In [58]:
df.groupBy('region').count().show()

+---------+-----+
|   region|count|
+---------+-----+
|northwest|  325|
|southeast|  364|
|northeast|  324|
|southwest|  325|
+---------+-----+



In [59]:
df.show()

+---+------+------+--------+------+---------+-----------+----------+----------------+
|age|   sex|   bmi|children|smoker|   region|    charges|first_name|     Region||BMI|
+---+------+------+--------+------+---------+-----------+----------+----------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|       new|  southwest 27.9|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523|       new| southeast 33.77|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|       new|  southeast 33.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|       new|northwest 22.705|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552|       new| northwest 28.88|
| 31|female| 25.74|       0|    no|southeast|  3756.6216|       new| southeast 25.74|
| 46|female| 33.44|       1|    no|southeast|  8240.5896|       new| southeast 33.44|
| 37|female| 27.74|       3|    no|northwest|  7281.5056|       new| northwest 27.74|
| 37|  male| 29.83|       2|    no|northeast|  6406.41

In [60]:
df.groupBy('region','smoker').count().show()

+---------+------+-----+
|   region|smoker|count|
+---------+------+-----+
|northeast|   yes|   67|
|southeast|    no|  273|
|southwest|    no|  267|
|southwest|   yes|   58|
|northwest|    no|  267|
|northeast|    no|  257|
|northwest|   yes|   58|
|southeast|   yes|   91|
+---------+------+-----+



In [61]:
df.drop('first_name').show()

+---+------+------+--------+------+---------+-----------+----------------+
|age|   sex|   bmi|children|smoker|   region|    charges|     Region||BMI|
+---+------+------+--------+------+---------+-----------+----------------+
| 19|female|  27.9|       0|   yes|southwest|  16884.924|  southwest 27.9|
| 18|  male| 33.77|       1|    no|southeast|  1725.5523| southeast 33.77|
| 28|  male|  33.0|       3|    no|southeast|   4449.462|  southeast 33.0|
| 33|  male|22.705|       0|    no|northwest|21984.47061|northwest 22.705|
| 32|  male| 28.88|       0|    no|northwest|  3866.8552| northwest 28.88|
| 31|female| 25.74|       0|    no|southeast|  3756.6216| southeast 25.74|
| 46|female| 33.44|       1|    no|southeast|  8240.5896| southeast 33.44|
| 37|female| 27.74|       3|    no|northwest|  7281.5056| northwest 27.74|
| 37|  male| 29.83|       2|    no|northeast|  6406.4107| northeast 29.83|
| 60|female| 25.84|       0|    no|northwest|28923.13692| northwest 25.84|
| 25|  male| 26.22|      

In [62]:
total_count=df.count()
print("Total Count: " + str(total_count))

_count = df.filter((col('region')=='southwest') & (col('children') ==3)).count()
print("sountwest Count: " + str(_count))

df.filter((col('region')=='southwest') & (col('children')==3 )).show()

Total Count: 1338
sountwest Count: 37
+---+------+----+--------+------+---------+-----------+----------+--------------+
|age|   sex| bmi|children|smoker|   region|    charges|first_name|   Region||BMI|
+---+------+----+--------+------+---------+-----------+----------+--------------+
| 53|female|28.1|       3|    no|southwest|  11741.726|       new|southwest 28.1|
| 54|female|30.8|       3|    no|southwest|   12105.32|       new|southwest 30.8|
| 30|female|30.9|       3|    no|southwest|   5325.651|       new|southwest 30.9|
| 32|  male|30.8|       3|    no|southwest|   5253.524|       new|southwest 30.8|
| 40|  male|35.3|       3|    no|southwest|   7196.867|       new|southwest 35.3|
| 27|  male|30.3|       3|    no|southwest|   4260.744|       new|southwest 30.3|
| 25|  male|26.8|       3|    no|southwest|   3906.127|       new|southwest 26.8|
| 43|female|34.4|       3|    no|southwest|   8522.003|       new|southwest 34.4|
| 35|  male|28.9|       3|    no|southwest|   5926.846|     

In [63]:
df.select('region').distinct().show()

+---------+
|   region|
+---------+
|northwest|
|southeast|
|northeast|
|southwest|
+---------+



In [64]:
df.select('region','age').distinct().show()

+---------+---+
|   region|age|
+---------+---+
|southeast| 26|
|southwest| 28|
|southeast| 55|
|southeast| 58|
|northeast| 38|
|southeast| 21|
|southeast| 46|
|southeast| 49|
|southwest| 44|
|southwest| 40|
|northeast| 18|
|northeast| 61|
|northwest| 59|
|southwest| 22|
|southwest| 56|
|southwest| 46|
|northwest| 36|
|southwest| 60|
|southeast| 57|
|southwest| 48|
+---------+---+
only showing top 20 rows



In [65]:
df.orderBy('age', ascending = False).show()

+---+------+------+--------+------+---------+-----------+----------+----------------+
|age|   sex|   bmi|children|smoker|   region|    charges|first_name|     Region||BMI|
+---+------+------+--------+------+---------+-----------+----------+----------------+
| 64|  male|  24.7|       1|    no|northwest|30166.61817|       new|  northwest 24.7|
| 64|female|  31.3|       2|   yes|southwest|  47291.055|       new|  southwest 31.3|
| 64|female| 39.33|       0|    no|northeast| 14901.5167|       new| northeast 39.33|
| 64|female|  33.8|       1|   yes|southwest|   47928.03|       new|  southwest 33.8|
| 64|  male|  34.5|       0|    no|southwest|  13822.803|       new|  southwest 34.5|
| 64|female|30.115|       3|    no|northwest|16455.70785|       new|northwest 30.115|
| 64|  male|  25.6|       2|    no|southwest|  14988.432|       new|  southwest 25.6|
| 64|female|32.965|       0|    no|northwest|14692.66935|       new|northwest 32.965|
| 64|  male| 39.16|       1|    no|southeast| 14418.28

In [67]:
df.groupBy('sex').count().orderBy('sex', ascending = False).show()

+------+-----+
|   sex|count|
+------+-----+
|  male|  676|
|female|  662|
+------+-----+



In [68]:
df1=spark.createDataFrame([[1,2,3,5]], ['c0','c1','c2','c4'])
df2=spark.createDataFrame([[5,9,3,0]], ['c0','c1','c2','c4'])
df1.union(df2).show()

+---+---+---+---+
| c0| c1| c2| c4|
+---+---+---+---+
|  1|  2|  3|  5|
|  5|  9|  3|  0|
+---+---+---+---+



In [69]:
df1 = spark.createDataFrame([[1,2,3]],['c0','c1','c2'])
df2 = spark.createDataFrame([[4,5,6]],['c0','c1','c2'])
df2.unionByName(df1).show()

+---+---+---+
| c0| c1| c2|
+---+---+---+
|  4|  5|  6|
|  1|  2|  3|
+---+---+---+

